In [222]:
import numpy as np
import pandas as pd

import os
import pickle
import glob
import matplotlib.pyplot as plt
import time
import keras
import imageio
from scipy import signal
from keras.models import load_model
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import explained_variance_score
from sklearn.metrics import roc_curve, auc
from skimage.transform import resize
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes


In [11]:
save_figures = False
file_ending = '.png'
model_string = 'NMDA'

dataset_folder = '../data'

models_folder     = os.path.join(dataset_folder, 'Models')
morphology_folder = os.path.join(dataset_folder, 'Morphology')
test_data_folder  = os.path.join(dataset_folder, 'Data_test')
#auxiliary_folder  = os.path.join(dataset_folder, 'Auxiliary')

model_filename           = os.path.join(models_folder, 'NMDA_TCN__DWT_7_128_153__model.h5')
model_metadata_filename  = os.path.join(models_folder, 'NMDA_TCN__DWT_7_128_153__training.pickle')
morphology_filename      = os.path.join(morphology_folder, 'morphology_dict.pickle')
#NN_illustration_filename = os.path.join(auxiliary_folder, 'TCN_7_layers.png')
test_files               = sorted(glob.glob(os.path.join(test_data_folder, '*_128_simulationRuns*_6_secDuration_*')))

print('-----------------------------------------------')
print('finding files: morphology and test data')
print('-----------------------------------------------')
#print('model found          : "%s"' %(model_filename.split('/')[-1]))
print('model metadata found : "%s"' %(model_metadata_filename.split('/')[-1]))
print('morphology found     : "%s"' %(morphology_filename.split('/')[-1]))
print('number of test files is %d' %(len(test_files)))
print('-----------------------------------------------')

-----------------------------------------------
finding files: morphology and test data
-----------------------------------------------
model metadata found : "NMDA_TCN__DWT_7_128_153__training.pickle"
morphology found     : "morphology_dict.pickle"
number of test files is 12
-----------------------------------------------


In [178]:
testing_3 = test_files[1]
experiment_dict = pickle.load(open(testing_3, "rb" ), encoding='latin1')


In [15]:
experiment_dict.keys()
df = pd.DataFrame(experiment_dict)

In [38]:
opening_files = test_files[:3]

In [180]:
def dataframe(file):
    experiment_dict = pickle.load(open(file, "rb"), encoding='latin1')
    df = pd.DataFrame(experiment_dict)
    df.drop('Results', inplace=True, axis=1)
    df = df.loc[['allSegmentsType', 'allSegmentsLength', 'allSegments_SectionInd', 'allSegments_DistFromSoma', 'allSegments_SectionDistFromSoma']]
    df = df_sub.T.apply(pd.Series.explode).reset_index()

    return df

#df_sub = df.loc[['allSectionsType', 'allSectionsLength', 'allSections_DistFromSoma']]

#df_sub.T.apply(pd.Series.explode).reset_index()
# stratify on column for basal/apical - use labelencoder

In [217]:
def multi_df(files):
    lst_df = []
    for file in files:
        lst_df.append(dataframe(file))
        
    return pd.concat(lst_df)

In [218]:
'''testing = test_files[:3]
dfs = multi_df(testing)''' #testing the definitions

In [220]:
dfs = multi_df(test_files)

In [225]:
le = LabelEncoder()
dfs['allSectionsType'] = le.fit_transform(dfs['allSectionsType'])
#1 = Basal
#0 = apical

In [224]:
dfs

,index,allSectionsType,allSectionsLength,allSections_DistFromSoma
0,Params,1,24.2009,11.5847
1,Params,1,15.1822,35.7856
2,Params,1,151.651,50.9678
3,Params,1,7.45563,50.9678
4,Params,1,92.5758,58.4234
...,...,...,...,...
188,Params,0,23.3443,57.7203
189,Params,0,93.6788,81.0646
190,Params,0,22.0852,81.0646
191,Params,0,36.0978,103.15


In [141]:
df
lst = list((df.loc['numOutputSpikesPerSim']))[0]

df1 = pd.DataFrame({'numOutputSpikesPerSim':lst})
df.Params.apply(lambda x: len(x))


numOutputSpikesPerSim                          128
allSegmentsLength                              639
allSegments_SectionInd                         639
allSegments_SectionDistFromSoma                639
allSectionsType                                193
allSectionsLength                              193
allSections_DistFromSoma                       193
allSegments_DistFromSoma                       639
allSegmentsType                                639
listOfISIs                                     494
inst_rate_sampling_time_interval_options_ms     18
Name: Params, dtype: int64